# Reporting Discovery - Deep Dive

> This builds on the early discovery work by attempting to consolidate both the > definition and instance trees to generate meaningful displayable data.
>


> As a reminder, these are the two main structures of our data hierarchy:
>
> ### Run-time(Instances)
> =18 -> 10 -> 19
>       +11 -> 09
>             +15
>             +08 -> 12
>                   +13
>                   +14
> ### Structural (Definitions)
> =07 -> 20 -> 01
>       +21
>       +06 -> 17
>       +08
>       +02
>       +05 -> 04 -> 16
> 
> In the previous study we used a library to simplify extraction of two-level > nested data, and developed an outline template of a recursive function that > could in theory build arbitrarily deep nested queries. We do not at this stage > know how meaningful very deep queries might be, but it is most likely that > edited versions of these would be very valuable in revealing information from > both the design-time and run-time data when filtered appropriately.
> 
> Because the algorithm has been successful, it is feasible to simply flesh out > the main recursive function to build the sql using the same reflective > techniques as where used in the two-layer nesting.
> 
> This will allow us to generate the queries needed to drive the main design-time > and run-time queries and decide how best to combine and separate these for best > usability.
> 
> Once the processor is working it should be incorporated in the raw library.

### Load the main library.

In [1]:
// Have modified the Raw library, so can reload that locally...
#i "file:///n:/nix-nuget/"
#r "nuget:nix-library-raw" 
#r "nuget:RazorInteractive" 

Restore sources file:///n:/nix-nuget/ Installed Packages nix-library-raw, 1.0.18 RazorInteractive, 1.2.0

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

### Connect the database

Using the library, 

- read the connection string from the environment
- create the JsonSqlClient
- as a test of connectivity, Get the column names from the rpt.reports table
- Display the result.

In [2]:
// some housekeeping needed to set up the library
using Nix.Library.Raw;
string connectionString = System.Environment.GetEnvironmentVariable("Nix_DB_DG");
JsonSqlClient sql = new JsonSqlClient(connectionString);

// verify that the libraries and the sql connection work
var x = sql.GetColumnNames("rpt.reports");
x.Display();

[ [ReportId], [OrganizationId], [ReportType], [FileNameTemplate], [Enabled], [Format], [TextQualifier], [TransmitterConfiguration], [Parameters], [ReportMethodId], [ReportTitle], [DateModified], [ReportInstanceId], [IsScheduledInApp] ]

## Definitions

To represent the two main divergent data paths through the metadata, we define an instanceData variable and a definitionData variable.

> The method used to generate the sql in the prior discovery used two fully qualified column names to get the rest of its information using refection.
>
> The method used here regards each related pair of column names as a single `Link`, and allows us to build arbitrary lengths of such links.
>
> This allows us to define an instanceData object and a definitionData object containing the two branches of the data hierarchy.

In [3]:
// for future work, these are the two main data sources
/*
=18 -> 10 -> 19
      +11 -> 09
            +15
            +08 -> 12
                  +13
                  +14
*/
// The data chain for investigating specific instances:
var instanceData = new Link[] {
    new Link("rpt.ReportTimeRanges.ReportTimeRangeId", "rpt.ReportInstances.TimePeriod", 2 ), //18
    new Link("rpt.Reports.ReportId", "rpt.ReportTransmitterConfigLinks.ReportId", 2 ), //10
    new Link("rpt.ReportTransmitterConfigs.ReportTransmitterConfigId", "rpt.ReportTransmitterConfigLinks.ReportTransmitterConfigId", 2), //19
    new Link("rpt.ReportInstances.ReportInstanceId", "rpt.Reports.ReportInstanceId", 2 ), //11
    new Link("rpt.Reports.ReportId", "rpt.ReportSchedule.ReportId", 2  ), //9
    new Link("rpt.ReportMethods.Id", "rpt.Reports.ReportMethodId", 2 ), //15
    new Link("rpt.Reports.ReportId", "rpt.ReportLogs.ReportId", 2 ), //8
    new Link("rpt.ReportLogs.ReportLogId", "rpt.ReportActivityLog.ReportLogId", 2 ), //12
    new Link("rpt.ReportLogs.ReportLogId", "rpt.ReportFileLog.ReportLogId", 2 ), //13
    new Link("rpt.ReportLogs.ReportLogId", "rpt.ReportTransmissionLogs.ReportLogId", 2 ), //14
};
/* //! todo: fix the main diagram to match whatever ends up wprking here.
=07 -> 20 -> 01
      +21
      +06 -> 17
      +08
      +02
      +05 -> 04 -> 16
      +00
      +03
      */ 
// the data chain for investigating definitions:
var definitionData = new Link[] {
    new Link ("rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportInstances.ReportDefinitionId", 2 ), //7
    new Link ("rpt.ReportTypes.ReportTypeId", "rpt.ReportDefinitions.ReportTypeId", 2 ), //20
    new Link ("rpt.ReportCategories.ReportCategoryId", "rpt.ReportTypes.ReportCategoryId", 2 ), //1
    new Link ("rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportDefinitionCustomAttributes.ReportDefinitionId", 2 ), //21
    new Link ("rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportDefinitionColumns.ReportDefinitionId", 2 ), //6
    new Link ("rpt.ReportSearchableTermTypes.ReportSearchableTermTypeId", "rpt.ReportDefinitionColumns.ReportSearchableTermTypeId", 2 ), //17
    new Link ("rpt.Reports.ReportId", "rpt.ReportLogs.ReportId", 2 ), //8
    new Link ("rpt.ReportColumns.ReportColumnId", "rpt.ReportDefinitionColumns.ReportColumnId", 2 ), //2
    new Link ("rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportColumns.ReportDataTypeId", 2 ), //3
    new Link ("rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportDefinitionColumns.ReportDataTypeIdOverride", 2 ), //5
    new Link ("rpt.ReportAggregateTypes.ReportAggregateTypeId", "rpt.ReportDefinitionColumns.ReportAggregateTypeId", 2 ), //0
    new Link ("rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportDataTypeOperators.ReportDataTypeId", 5 ), //4
    new Link ("rpt.ReportOperatorTypes.ReportoperatorTypeId", "rpt.ReportDataTypeOperators.ReportOperatorTypeId", 2 ), //16
};


### Publishing for the Discovery of Instances and Definitions 

Having defined the links, we can use the GenerateNestedQuery method to make sql, which we can then apply to generate Json data, which we can then pass to a razor page for local display.

> For convenience we can save the sql and the json for later off-line use
> 
> The sql generator also generates a diagnostic output that describes the way the link sequence was interpretted.  This can be useful for diagnosing weird behavior from specifying the wrong start node, for example.
> 
> We display that here as a snity check.

In [4]:
using Nix.Library.Raw;

List<string> Pages = new List<string>();

Nest.ToFile(Nest.GetCss(), "../.data/deep-dive.css");
// Definitions: generate and save sql
var definitionSql = sql.GenerateNestedQuery(definitionData);
definitionSql[1].Display();
Nest.ToFile(definitionSql[0], "../.data/definitions-raw.sql");
// generate and save json
var definitionJson = sql.JsonSingleUsingQuery(definitionSql[0]);
Nest.ToFile(definitionJson, "../.data/definitions-raw.json");
// insstances: cregenerate and save sql
var instanceSql = sql.GenerateNestedQuery(instanceData);
instanceSql[1].Display();
Nest.ToFile(instanceSql[0], "../.data/instances-raw.sql");
// generate and svae json
var instanceJson = sql.JsonSingleUsingQuery(instanceSql[0]);
Nest.ToFile(instanceJson, "../.data/instances-raw.json");



Root: -- ReportDefinitions <+--++ ReportInstances --
Peer: -- ReportTypes <+--++ ReportDefinitions --
Nest: -- ReportTypes <+--++ ReportCategories --
Peer: -- ReportDefinitionCustomAttributes <+--++ ReportDefinitions --
Peer: -- ReportDefinitionColumns <+--++ ReportDefinitions --
Nest: -- ReportDefinitionColumns <+--++ ReportSearchableTermTypes --
Peer: -- ReportColumns <+--++ ReportDefinitionColumns --
Nest: -- ReportColumns <+--++ ReportDataTypes --
Nest: -- ReportDataTypes <+--++ ReportDefinitionColumns --
Nest: -- ReportDefinitionColumns <+--++ ReportAggregateTypes --
Peer: -- ReportDataTypeOperators <+--++ ReportDataTypes --
Nest: -- ReportDataTypeOperators <+--++ ReportOperatorTypes --

Root: -- ReportTimeRanges <+--++ ReportInstances --
Nest: -- ReportInstances <+--++ Reports --
Nest: -- Reports <+--++ ReportTransmitterConfigLinks --
Nest: -- ReportTransmitterConfigLinks <+--++ ReportTransmitterConfigs --
Peer: -- ReportSchedule <+--++ Reports --
Peer: -- ReportMethods <+--++ Reports --
Peer: -- ReportLogs <+--++ Reports --
Nest: -- ReportLogs <+--++ ReportActivityLog --
Peer: -- ReportFileLog <+--++ ReportLogs --
Peer: -- ReportTransmissionLogs <+--++ ReportLogs --

In [5]:
// rendering these from file allows us to repeat this step on its own while refining the presentation settings which we likewise save to a file
Nest.NestOptions options = new Nest.NestOptions
{
    DarkMode = false,
    MaximumCellPixelWidth = 10000,
    MaximumPixelWidth = 10000,
    TitleColumnPixelWidth = 120
};
var y = Nest.GetCss(options);
//x.Display();
Nest.ToFile(y, "../.html/deep-dive.css");
Pages.Add(Nest.RenderHtmlStepping(Nest.FromFile("../.data/definitions-raw.json"), "Definitions", 10, "Sample Report Definitions"));
Pages.Add(Nest.RenderHtmlStepping(Nest.FromFile("../.data/instances-raw.json"), "Instances", 10, "Sample Report Instances"));

Nest.ToFile(Nest.RenderPageStepping(definitionJson, "DEFS", 5, "Report Definitions", options), "./Definitions.html");
Nest.ToFile(Nest.RenderPageStepping(instanceJson, "DEFS", 5, "Report Instances", options), "./Instances.html");
"Written to file".Display();

Nest.RunFile(".\\Definitions.html");
Nest.RunFile(".\\Instances.html");
"Launched in browser".Display();

Written to file

Launched in browser

### Quick view using Razor

The pages can be also viewed in a Razor page for a quick check. This has some limitations, for example the steppingf version does not execute live here so we use a scrolling version of the page (not yet fully investigated).

Html can also be pasted into an html cell though, and runs in the notebook.

In [6]:
Nest.ToFile(Nest.RenderPageScrollin(definitionJson, "DEFS", 5, "Report Definitions", options), "./Definitions.html");
Nest.ToFile(Nest.RenderPageScrolling(instanceJson, "DEFS", 5, "Report Instances", options), "./Instances.html");

Nest.ToFile(Nest.RenderPageScrollin(definitionJson, "DEFS", 5, "Report Definitions", options), "./Definitions.html");
Nest.ToFile(Nest.RenderPageScrolling(instanceJson, "DEFS", 5, "Report Instances", options), "./Instances.html");

In [7]:
#!razor

<html>
  <head>
    <link rel="stylesheet" href="./deep-dive.css">
  </head>
  <body>
        @Raw(@Model.Pages[0])
        <hr>
        @Raw(@Model.Pages[1])
      <hr>
  </body>
</html>

Sample Report Definitions Definitions 1 elements ReportDefinitions 
 1/2 2/2 ReportDefinitionId 32 31 ReportTypeId 32 31 CreatedDate 2024-01-29T17:11:12.2229288 2023-09-19T16:12:38.4975525 ModifiedDate ReportInstance ReportInstance 
 1/5 2/5 3/5 4/5 5/5 ReportInstanceId 5764 5765 5960 6284 6285 ReportDefinitionId 32 32 32 32 32 ReportTypeId 32 32 32 32 32 OrganizationId 1 1 1 1 1 IsSavedPreset false false false false false PresetTitle Skill Standard (Detailed by User) Skill Standard (Detailed by User) Skill Standard (Detailed by User) Skill Standard (Detailed by User) Skill Standard (Detailed by User) TimePeriod 1 1 1 1 1 CustomStartDate CustomEndDate FilteredData {"ReportDefinitionId":32,"VisibleColumns":[{"Id":"main_489","ReportDefinitionColumnId":489,"UseAggregate":false,"Order":1},{"Id":"main_501","ReportDefinitionColumnId":501,"UseAggregate":false,"Order":2},{"Id":"main_495","ReportDefinitionColumnId":495,"UseAggregate":false,"Order":3},{"Id":"main_492","ReportDefinitionColumnId":492,"UseAggregate":false,"Order":4},{"Id":"main_498","ReportDefinitionColumnId":498,"UseAggregate":false,"Order":5},{"Id":"main_493","ReportDefinitionColumnId":493,"UseAggregate":false,"Order":6},{"Id":"main_502","ReportDefinitionColumnId":502,"UseAggregate":false,"Order":7},{"Id":"main_490","ReportDefinitionColumnId":490,"UseAggregate":false,"Order":8},{"Id":"main_494","ReportDefinitionColumnId":494,"UseAggregate":false,"Order":9},{"Id":"main_500","ReportDefinitionColumnId":500,"UseAggregate":false,"Order":10},{"Id":"main_491","ReportDefinitionColumnId":491,"UseAggregate":false,"Order":11},{"Id":"main_496","ReportDefinitionColumnId":496,"UseAggregate":false,"Order":12},{"Id":"main_503","ReportDefinitionColumnId":503,"UseAggregate":false,"Order":13},{"Id":"main_497","ReportDefinitionColumnId":497,"UseAggregate":false,"Order":14},{"Id":"main_499","ReportDefinitionColumnId":499,"UseAggregate":false,"Order":15},{"Id":"main_504","ReportDefinitionColumnId":504,"UseAggregate":false,"Order":16}],"TimeRange":{"Id":"-1","ReportDefinitionColumnId":0,"IsCustom":false,"ReportTimeRangeId":1,"StartDateTime":null,"EndDateTime":null},"FilterColumns":[{"Id":"-1","ReportDefinitionColumnId":0,"DataTypeOperatorId":0,"IsForTimeRange":true}],"ConditionGroups":[],"CustomAttributeColumns":[],"SegmentIds":[]} {"ReportDefinitionId":32,"VisibleColumns":[{"Id":"main_489","ReportDefinitionColumnId":489,"UseAggregate":false,"Order":1},{"Id":"main_501","ReportDefinitionColumnId":501,"UseAggregate":false,"Order":2},{"Id":"main_495","ReportDefinitionColumnId":495,"UseAggregate":false,"Order":3},{"Id":"main_492","ReportDefinitionColumnId":492,"UseAggregate":false,"Order":4},{"Id":"main_498","ReportDefinitionColumnId":498,"UseAggregate":false,"Order":5},{"Id":"main_493","ReportDefinitionColumnId":493,"UseAggregate":false,"Order":6},{"Id":"main_502","ReportDefinitionColumnId":502,"UseAggregate":false,"Order":7},{"Id":"main_490","ReportDefinitionColumnId":490,"UseAggregate":false,"Order":8},{"Id":"main_494","ReportDefinitionColumnId":494,"UseAggregate":false,"Order":9},{"Id":"main_500","ReportDefinitionColumnId":500,"UseAggregate":false,"Order":10},{"Id":"main_491","ReportDefinitionColumnId":491,"UseAggregate":false,"Order":11},{"Id":"main_496","ReportDefinitionColumnId":496,"UseAggregate":false,"Order":12},{"Id":"main_503","ReportDefinitionColumnId":503,"UseAggregate":false,"Order":13},{"Id":"main_497","ReportDefinitionColumnId":497,"UseAggregate":false,"Order":14},{"Id":"main_499","ReportDefinitionColumnId":499,"UseAggregate":false,"Order":15},{"Id":"main_504","ReportDefinitionColumnId":504,"UseAggregate":false,"Order":16}],"TimeRange":{"Id":"-1","ReportDefinitionColumnId":0,"IsCustom":false,"ReportTimeRangeId":1,"StartDateTime":null,"EndDateTime":null},"FilterColumns":[],"ConditionGroups":[],"CustomAttributeColumns":[],"SegmentIds":[]} {"ReportDefinitionId":32,"VisibleColumns":[{"Id":"main_489","ReportDefinitionColumnId":489,"UseAggregate":false,"Order":1},{"Id":"main_501","Rep

### And we can save the files and launch them in the browser.

Here let's render to complete files and launch them in the browser.
